# scanpy 10X single cell RNAseq workbook

In [ ]:
# Jupyter Notebook cell

# In [1]: Install or load required packages:
# !pip install scanpy matplotlib seaborn
# (You may also install in a conda environment: conda install -c conda-forge scanpy seaborn)

import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# In [2]: Set up some basic Scanpy configurations and read 10x data

# Scanpy settings (adjust as you like)
sc.settings.verbosity = 2           # Show info on progress
sc.logging.print_header()
sc.settings.set_figure_params(dpi=100, facecolor='white')

# Path to your 10x filtered_feature_bc_matrix directory
data_dir = "results/cellranger_count/SAMPLE_1/outs/filtered_feature_bc_matrix/"

# Read 10x data (this automatically looks for matrix.mtx.gz, features.tsv.gz, barcodes.tsv.gz)
# var_names='gene_symbols' will attempt to load gene names from the second column of features.tsv
adata = sc.read_10x_mtx(
    data_dir, 
    var_names='gene_symbols', 
    make_unique=True
)

adata.var_names_make_unique()  # Ensures unique gene names

print(adata)

In [ ]:
# In [3]: Basic QC metrics

# Calculate the total counts per cell and number of genes per cell
adata.obs['n_counts'] = adata.X.sum(axis=1).A1 if hasattr(adata.X, 'A') else adata.X.sum(axis=1)
adata.obs['n_genes'] = (adata.X > 0).sum(axis=1).A1 if hasattr(adata.X, 'A') else (adata.X > 0).sum(axis=1)

# Calculate mitochondrial gene percentage
# For human data: mitochondrial genes often start with 'MT-'
mito_genes = [name for name in adata.var_names if name.startswith('MT-')]
# Calculate the fraction of counts in mito genes
adata.obs['percent_mt'] = (
    adata[:, mito_genes].X.sum(axis=1).A1 / adata.obs['n_counts']
) * 100 if len(mito_genes) > 0 else 0

# Show violin plots of QC metrics
sc.pl.violin(adata, ['n_genes', 'n_counts', 'percent_mt'], jitter=0.4, multi_panel=True)

In [ ]:
# In [4]: Filter cells and genes

# 1) Remove genes not expressed in at least 3 cells
sc.pp.filter_genes(adata, min_cells=3)

# 2) Remove cells with fewer than 200 detected genes
sc.pp.filter_cells(adata, min_genes=200)

# 3) Filter out cells with too many genes (possible doublets) or too high mitochondrial ratio
#    Adjust thresholds based on your dataset distribution
adata = adata[adata.obs['n_genes'] < 2500, :]
adata = adata[adata.obs['percent_mt'] < 5, :]

print(f"Remaining cells: {adata.n_obs}, Remaining genes: {adata.n_vars}")

In [ ]:
# In [5]: Normalize and log-transform

# Normalize each cell by total counts, multiply by 1e4
sc.pp.normalize_total(adata, target_sum=1e4)

# Log-transform the data
sc.pp.log1p(adata)

# Identify highly variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

# Keep only highly variable genes for downstream steps
adata = adata[:, adata.var['highly_variable']]

In [ ]:
# In [6]: Scale data and run PCA

# Scale each gene to unit variance (with max_value to clip large values)
sc.pp.scale(adata, max_value=10)

# PCA
sc.tl.pca(adata, svd_solver='arpack')

# Plot the variance ratio to help decide how many PCs to use
sc.pl.pca_variance_ratio(adata, log=True)

In [ ]:
# In [7]: Neighbors, clustering, and UMAP

# Choose ~10–15 PCs based on the elbow plot
n_pcs = 10

# Calculate nearest neighbors
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=n_pcs)

# Compute UMAP
sc.tl.umap(adata)

# Clustering (Leiden algorithm by default)
sc.tl.leiden(adata, resolution=0.5)

# UMAP plot colored by cluster
sc.pl.umap(adata, color=['leiden'], legend_loc='on data')

In [ ]:
# In [8]: Visualize QC metrics on UMAP

# Sometimes it's helpful to color by total counts or mitochondrial percentage
sc.pl.umap(adata, color=['n_counts', 'percent_mt'])

In [ ]:
# In [9]: Identify marker genes (optional, can be expensive)

# If you want to find markers for each cluster:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes=5, sharey=False)

In [ ]:
# In [10]: Summary
# We have performed:
#   1. QC and filtering
#   2. Normalization and log transformation
#   3. Identification of highly variable genes
#   4. PCA and UMAP
#   5. Clustering (Leiden)
#   6. Marker gene identification (optional)

# Check the final adata object
adata

In [ ]:
# Save the AnnData object to an .h5ad file
adata.write("results/processed_sc_data.h5ad")

# To ensure you can load it later, you can test as follows:
# import scanpy as sc
# adata_loaded = sc.read("results/processed_sc_data.h5ad")
# adata_loaded

Key Steps Recap

    Read 10X output (barcodes.tsv.gz, features.tsv.gz, matrix.mtx.gz) into a Scanpy AnnData object.
    QC: Filter out low-quality cells and undesired genes:
        Minimum #genes per cell, maximum #genes, maximum % mitochondrial reads, etc.
    Normalization: sc.pp.normalize_total(…) to ensure each cell has the same total read count.
    Log transform: sc.pp.log1p(…).
    Find highly variable genes and subset the data to those genes.
    Scale the data and run PCA to reduce dimensionality.
    Compute nearest neighbors and run UMAP for visualization.
    Cluster (Leiden) to identify cell subpopulations.
    Identify marker genes per cluster if needed.

You can further refine your analysis by:

    Trying different mitochondrial or gene-count thresholds.
    Using Doublet detection (e.g., scrublet or Scanpy–wrapped methods).
    Integrating multiple samples with Scanpy integration tools.
    Annotating clusters with known or predicted marker genes for your organism/tissue of interest.

With Scanpy, you have a flexible Python-based ecosystem to perform end-to-end single-cell analyses in a Jupyter notebook environment. Enjoy exploring your single-cell RNA-seq data!